In [13]:
import numpy as np
import pandas as pd
from utility import data_loader, data_cleaner, sample_dist
from cmdstanpy import CmdStanModel, write_stan_json
import arviz as az

In [3]:
berlin_data, vanautu_data, adult_data, child_data = data_loader()

In [4]:
d_berlin, d_vanautu, MA, pop_berlin, pop_berlin_raw, pop_vanautu_raw, pop_vanautu = data_cleaner(berlin_data, vanautu_data, adult_data, child_data)

/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/utility.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_data["choice"] = adult_data["T1_ad_choice_1yes"]
/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/utility.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_data["choice"] = child_data["T1_choice_1yes"]
/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/utility.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [5]:
# Create matrices with zeros
sample_berlin, sample_vanuatu = sample_dist(d_berlin, d_vanautu)

In [6]:
#Covaraince Kenrel defined for the paper and implemented in Stan.
MA = 20
age = d_berlin["Age_binned"].values - 1
outcome = d_berlin["choice"]
gender = d_berlin["gender"].values - 1
d_berlinf = d_berlin[d_berlin["gender"] == 1]
d_berlinm =d_berlin[d_berlin["gender"] == 2]
d_berlinf['Age_binned'] = d_berlinf["Age_binned"].astype(int)
d_berlinm['Age_binned'] = d_berlinm["Age_binned"].astype(int)

/tmp/ipykernel_213409/3536775231.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_berlinf['Age_binned'] = d_berlinf["Age_binned"].astype(int)
/tmp/ipykernel_213409/3536775231.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_berlinm['Age_binned'] = d_berlinm["Age_binned"].astype(int)


In [7]:
model = CmdStanModel(stan_file="model_MRpoststratification.stan", cpp_options={'STAN_THREADS': 'TRUE'},
                     force_compile= True)

21:16:30 - cmdstanpy - INFO - compiling stan file /home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/model_MRpoststratification.stan to exe file /home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/model_MRpoststratification
21:17:07 - cmdstanpy - INFO - compiled model executable: /home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/model_MRpoststratification


In [27]:
databerlin = {"N": len(d_berlin),              #Sample size
                      "MA":20,                         #Number of age categories
                      "age": d_berlin["Age_binned"],       #Binned ages
                      "outcome": d_berlin["choice"],       #Choices in dictator game
                      "gender": d_berlin["gender"],        #Gender
                      "P_Pop": np.transpose(pop_berlin_raw),       #Population demography of Berlin
                      "P_other": np.transpose(pop_vanautu_raw)}
datavanautu = {"N": len(d_vanautu),              #Sample size
                      "MA":20,                         #Number of age categories
                      "age": d_vanautu["Age_binned"],       #Binned ages
                      "outcome": d_vanautu["choice"],       #Choices in dictator game
                      "gender": d_vanautu["gender"],        #Gender
                      "P_Pop": np.transpose(pop_berlin_raw),       #Population demography of Berlin
                      "P_other": np.transpose(pop_vanautu_raw)}

In [31]:
write_stan_json("/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/data_b.json", data = databerlin)
write_stan_json("/home/harrison/Desktop/gitHubRepos/CausalAnalysis/generalisability/data_v.json", data = datavanautu)

In [32]:
fitb = model.sample("data_b.json", chains = 4 , iter_sampling=1000, max_treedepth = 13, adapt_delta = .99, seed = 1, parallel_chains = 4)
fitv = model.sample("data_v.json", chains = 4 , iter_sampling=1000, max_treedepth = 13, adapt_delta = .99, seed = 1, parallel_chains = 4)

21:46:40 - cmdstanpy - INFO - CmdStan start processing
chain 1 |                                                         | 00:00 Status
chain 2 |                                                         | 00:00 Status

chain 3 |                                                         | 00:00 Status


chain 4 |                                                         | 00:00 Status
chain 1 |██▏                     | 00:01 Iteration:    1 / 2000 [  0%]  (Warmup)


chain 4 |██▏                     | 00:01 Iteration:    1 / 2000 [  0%]  (Warmup)

chain 3 |██▌                                                      | 00:02 Status
chain 1 |███▎                    | 00:04 Iteration:  100 / 2000 [  5%]  (Warmup)

chain 3 |██▏                     | 00:04 Iteration:  100 / 2000 [  5%]  (Warmup)
chain 2 |████▎                   | 00:04 Iteration:  200 / 2000 [ 10%]  (Warmup)

chain 3 |███▎                    | 00:05 Iteration:  200 / 2000 [ 10%]  (Warmup)
chain 1 |████▎                   | 00:05 Itera


21:52:44 - cmdstanpy - INFO - CmdStan done processing.
21:52:44 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 4 had 77 iterations at max treedepth (7.7%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.
21:52:44 - cmdstanpy - INFO - CmdStan start processing


chain 1 |                                                         | 00:00 Status
chain 2 |                                                         | 00:00 Status

chain 3 |                                                         | 00:00 Status


chain 4 |                                                         | 00:00 Status


chain 4 |██▏                     | 00:01 Iteration:    1 / 2000 [  0%]  (Warmup)

chain 3 |██▏                     | 00:02 Iteration:    1 / 2000 [  0%]  (Warmup)
chain 1 |██▏                     | 00:05 Iteration:    1 / 2000 [  0%]  (Warmup)


chain 4 |███▎                    | 00:05 Iteration:  100 / 2000 [  5%]  (Warmup)


chain 4 |████▎                   | 00:06 Iteration:  200 / 2000 [ 10%]  (Warmup)

chain 3 |███▎                    | 00:06 Iteration:  100 / 2000 [  5%]  (Warmup)


chain 4 |█████▍                  | 00:09 Iteration:  300 / 2000 [ 15%]  (Warmup)
chain 1 |███▎                    | 00:10 Iteration:  100 / 2000 [  5%]  (Warmup)

chain 3 |████▎


22:01:53 - cmdstanpy - INFO - CmdStan done processing.
22:01:53 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: Exception: cholesky_decompose: Matrix m is not positive definite (in 'model_MRpoststratification.stan', line 15, column 3 to column 36) (in 'model_MRpoststratification.stan', line 53, column 3 to column 99)
Consider re-running with show_console=True if the above output is unclear!
22:01:53 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 1 iterations at max treedepth (0.1%)
	Chain 3 had 22 iterations at max treedepth (2.2%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [33]:
fitb.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-56.553000,3.163070,26.344300,-92.837200,-59.800500,-4.092520,69.3674,0.183108,1.05539
alpha[1],-0.061394,0.079832,1.556390,-2.735840,-0.079022,2.645500,380.0900,1.003320,1.00725
alpha[2],0.552491,0.107205,1.838050,-2.482360,0.486664,3.636940,293.9560,0.775951,1.01590
"age_effect[1,1]",-0.549198,0.085254,1.725810,-3.783800,-0.306093,2.116750,409.7880,1.081710,1.00753
"age_effect[1,2]",-0.581188,0.083839,1.600930,-3.516410,-0.375984,1.922500,364.6270,0.962497,1.00860
...,...,...,...,...,...,...,...,...,...
pred_p_f[16],0.825972,0.006541,0.186092,0.439824,0.896668,0.997997,809.3960,2.136550,1.00348
pred_p_f[17],0.830106,0.006664,0.189667,0.426753,0.901154,0.998581,810.1640,2.138570,1.00469
pred_p_f[18],0.830364,0.006749,0.192346,0.425411,0.908978,0.998889,812.1830,2.143900,1.00312
pred_p_f[19],0.827815,0.006391,0.198367,0.412444,0.912424,0.998955,963.3720,2.542990,1.00481


In [37]:
fitv.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-32.310800,7.086910,34.402400,-85.480100,-32.984400,29.033800,23.5647,0.040882,1.17663
alpha[1],-0.261610,0.033813,0.730542,-1.393020,-0.271825,0.996607,466.8040,0.809849,1.00401
alpha[2],-0.808029,0.047111,0.884022,-2.097280,-0.862681,0.657848,352.1180,0.610883,1.01223
"age_effect[1,1]",-0.066359,0.033815,0.721667,-1.370960,-0.012278,0.990449,455.4610,0.790170,1.00470
"age_effect[1,2]",-0.017086,0.034486,0.712902,-1.254050,-0.000617,1.084750,427.3370,0.741378,1.00386
...,...,...,...,...,...,...,...,...,...
pred_p_f[16],0.349546,0.011482,0.145817,0.180495,0.316858,0.663732,161.2930,0.279824,1.03444
pred_p_f[17],0.351752,0.012197,0.150320,0.180220,0.317169,0.673486,151.8880,0.263508,1.03572
pred_p_f[18],0.354479,0.012535,0.152958,0.180586,0.318863,0.679944,148.9080,0.258338,1.03507
pred_p_f[19],0.357357,0.013461,0.161389,0.174752,0.316078,0.720176,143.7390,0.249371,1.03540


In [35]:
fitb_df = fitb.draws_pd()

In [36]:
arviz_varb = az.from_cmdstanpy(posterior = fitb)